# Experiencia SEDICI 

## Detección automática de repeticiones parciales en repositorios digitales

El repositorio del proyecto es [Github](https://github.com/germanchaca/OAI-PMH-ABSTRACTS_PARTIAL_REPETITIONS-).


### Alcance
* Se extraerán los metadatos de un proveedor de datos OAI-PMH y cómo convertir los datos recuperados de Dublin Core en un pandas dataframe. Se podrá utilizar le código en cualquiera de los proveedores de metadatos listados en buscadores como [sherpa](https://v2.sherpa.ac.uk/view/repository_by_country/Argentina.html) o [roar](http://roar.eprints.org/).

* Se llevará a cabo un preprocesamiento de textos para su posterior explotación.
* Finalmente, se bucará detectar repeticiones parciales en el texto del abstract o resumen de los registros publicados en el repositorio SEDICI. EL código puede ser utilizado para detectar repeticiones parciales entre los distintos repositorios ditigales que soporten la interoperabilidad de metadatoscon OAI-PMH.


### Preparación del entorno de trabajo
El siguiente fragmento de código inicializa su entorno de tiempo de ejecución de Python para ejecutar todas las acciones posteriores.

Si ha instalado todo correctamente con la ayuda de [Anaconda](https://www.continuum.io/downloads), los siguientes paquetes deben instalarse correctamente y ser importados sin problemas.

In [ ]:
!pip install jellyfish
!pip install geojson
!pip install geopy
!pip install sickle
!pip install googlemaps

### Sickle - Librería para el manejo de OAI-PMH

[Sickle](https://sickle.readthedocs.io/en/latest/) es una biblioteca de OAI-PMH escrita en Python. Ha sido diseñado para recuperar datos cómodamente de OAI interactúa a la manera Pythonic

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time

pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

from collections import OrderedDict # provides the ordered dictionary
import re # for regular expressions used below
import urllib # to read from URLs
import json
import itertools
import os.path
from datetime import datetime # for time measurement
import sys
import os
import pickle
import subprocess as subp
import gzip
import math

import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
from IPython.display import display
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
# OAI
from sickle import Sickle

allowDownloads=True # should be True if you run this for the first time
downloadDir="./tmp/"
allowExactStringMatching=True

def printLog(text):
    now=str(datetime.now())
    print("["+now+"]\t"+text)
    # forces to output the result of the print command immediately, see: http://stackoverflow.com/questions/230751/how-to-flush-output-of-python-print
    sys.stdout.flush()
    
def pickleCompress(fileName,pickledObject):
    printLog("Pickling to '%s'" %fileName)
    f = gzip.open(fileName,'wb')
    pickle.dump(pickledObject,f)
    f.close()
    printLog("Pickling done.")
    
def pickleDecompress(fileName):
    #restore the object
    printLog("Depickling from '%s'" %fileName)
    f = gzip.open(fileName,'rb')
    pickledObject = pickle.load(f)
    f.close()
    printLog("Depickling done.")
    return pickledObject

if not os.path.exists(downloadDir):
        os.makedirs(downloadDir)
if not os.path.exists("./picklez/"):
        os.makedirs("./picklez/")     

## Recolección de metadatos de un repositorio digital

### Conexión al repositorio proveedor OAI y descarga de registros de metadatos

La siguiente celda se conecta al servidor OAI-PMH de Repositorio de la Universidad Nacional de La Plata - [SEDICI](http://sedici.unlp.edu.ar/).

los registros se guardan localmente para trabajar con ellos más tarde. Por simplicidad, el ahorro se basa en el mecanismo de pickling de Python, su método de serialización de objetos incorporado. En un caso de uso productivo, es posible que alcance los límites del paquete pickle debido al tamaño del archivo resultante. Una alternativa viable es [HDF5](https://www.h5py.org/).

In [2]:
repository_address = 'http://sedici.unlp.edu.ar/oai/snrd'
allowDownloads = True
#repository_address = 'http://repositorio.filo.uba.ar/oai/request'

Se listan los conjuntos o sets de datos proporcionados por el proveedor de datos

In [8]:
sickle = Sickle(repository_address)
sets = sickle.ListSets()
print ("Conjuntos proporcionados por el proveedor de datos\n* * * * * * * * * * * * * * * * * * * * * ") 
i = 0
for s in sets:
    print("'"+s.setName+"' accessible via: '"+s.setSpec+"'")
    i = i + 1
    if i > 20: break

Conjuntos proporcionados por el proveedor de datos
* * * * * * * * * * * * * * * * * * * * * 
'Conjunto de recursos para Sistema Nacional de Repositorios Digitales, Argentina SNRD' accessible via: 'snrd'
'Unidades académicas' accessible via: 'com_10915_1'
'Facultad de Ingeniería' accessible via: 'com_10915_2'
'Facultad de Ciencias Veterinarias' accessible via: 'com_10915_5'
'Facultad de Ciencias Jurídicas y Sociales' accessible via: 'com_10915_7'
'Facultad de Ciencias Agrarias y Forestales' accessible via: 'com_10915_10'
'Facultad de Periodismo y Comunicación Social' accessible via: 'com_10915_12'
'Facultad de Ciencias Astronómicas y Geofísicas' accessible via: 'com_10915_15'
'Facultad de Trabajo Social' accessible via: 'com_10915_18'
'Facultad de Odontología' accessible via: 'com_10915_20'
'Facultad de Ciencias Exactas' accessible via: 'com_10915_22'
'Facultad de Arquitectura y Urbanismo' accessible via: 'com_10915_25'
'Facultad de Psicología' accessible via: 'com_10915_28'
'Facultad 

### Análisis sobre la colección 'Facultad de Informática'

Si se quiere recuperar los registros de un conjunto/set específico se debe indicar el parámetro set, de lo contrario se listarán todos los registros disponibles.

In [9]:
maxDocs=100000
# Si vuelve a ejecutar esta celda, el contenido de la matriz SavedRecords se modificará.
# 2:15 h para 100k

savedRecords=[]
if allowDownloads:
    sickle = Sickle(repository_address)
    # using Dublin Core format 
    #records = sickle.ListRecords(metadataPrefix='oai_dc')
    records = sickle.ListRecords(metadataPrefix='oai_dc', set = 'com_10915_36')
    printLog("Inicio de la descarga de registros OAI-PMH...")
    savedDocs=0
    for record in records:
        # get the PPN of the digitized work as the DC identifier will reference different (analog) manifestations
        tokens=record.header.identifier.split(":")
        tokens
        
        if savedDocs<maxDocs:
            savedDocs=savedDocs+1
            savedRecords.append(record.metadata)
            if savedDocs%1000==0:
                printLog("Downloaded %d of %d records."%(savedDocs,maxDocs))
        else:
            break
    printLog("Descarga finalizada de OAI-PMH de "+str(len(savedRecords))+" registros.")
    pickle.dump( savedRecords, open( "sedici_dc_all.pickle", "wb" ) )
else:
    printLog("Cargando registros OAI-PMH desde el disco...")
    savedRecords=pickle.load( open( "sedici_dc_all.pickle", "rb" ) )
    printLog("Done.")

[2020-08-12 21:55:15.706000]	Inicio de la descarga de registros OAI-PMH...
[2020-08-12 21:55:21.762000]	Downloaded 1000 of 100000 records.
[2020-08-12 21:55:23.802000]	Descarga finalizada de OAI-PMH de 1255 registros.


In [10]:
record = savedRecords[34]
record["identifier"]

['http://sedici.unlp.edu.ar/handle/10915/3864']

In [11]:
record

{'contributor': ['Bertone, Rodolfo Alfredo', u'Pessacq, Ra\xfal Adolfo'],
 'creator': [u'Beneforti, Mar\xeda Fernanda', u'Ainchil, Mar\xeda Virginia'],
 'date': ['2000'],
 'description': [u'El presente trabajo de grado surgi\xf3 para canalizar inquietudes respecto del uso de la computadora en el aprendizaje. El objetivo inicial tuvo su origen en la idea de realizar un aporte para lograr mejorar el nivel de lectura y escritura de los estudiantes secundarios, en una forma amena, \xe1gil, con un medio atractivo o usual para ellos, como es la computadora; mediante algunos de los mejores textos literarios de ficci\xf3n de habla espa\xf1ola, su an\xe1lisis e interpretaci\xf3n. Las t\xe9cnicas que se han utilizado para llevar a cabo desarrollos inform\xe1ticos orientados al aprendizaje han sido diversas, siendo la hipermedia una de las que goza de mayor aceptaci\xf3n. Esta tecnolog\xeda proporciona caracter\xedsticas que resultan muy \xfatiles en el campo del aprendizaje por computadora, como

### Crear un pandas dataframe a partir de los registros de metadatos

In [12]:
# load the records
printLog("Cargando registros...")
# uncomment this line if you are continuing the execution of this notebook at a later point in time
savedRecords=pickle.load( open( "sedici_dc_all.pickle", "rb" ) )
printLog("Finalizada la carga de registros.")

availableKeys=dict()

# check for all keys present in the previously downloaded dataset
for i,r in enumerate(savedRecords):
    for k in r.keys():
        if not k in availableKeys:
            availableKeys[k]=1
        else:
            availableKeys[k]=availableKeys[k]+1
    
print(availableKeys)

# create a dictionary for the records
values=dict()
# take the keys as they have found within the downloaded OAI records
keys=availableKeys.keys()
# for every metadata field, create an empty array as the content of the dictionary filed under the key 'k'
for k in keys:
    values[k]=[]

# iterate over all saved records
for record in savedRecords:
    # we cannot iterate over the keys of record.metadata directly because not all records cotain the same fields,...
    for k in keys:
        # thus we check if the metadata field 'k' has been created above
        if k in values:
            # append the metadata fields to the dictionary created above
            # if the metadata field 'k' is not available input "None" instead
            #values[k].append(record.get(k,["None"])[0].encode('ISO-8859-1'))
            if k in record:
                value=record.get(k)[0]
                if value:
                    if value.isdigit():
                        value=int(value)
                    else:
                        #p27 value=value.encode('ISO-8859-1')
                            #value=value.encode('ISO-8859-1').decode("utf-8", "backslashreplace")
                        value=value
                    values[k].append(value)
                else:
                    values[k].append(np.nan)
            else:
                values[k].append(np.nan)
# create a data frame from the 
#p27 df=pd.DataFrame(pd.to_numeric(values,errors='coerce'))
df=pd.DataFrame(values)
df['date']=pd.to_numeric(df['date'],errors='ignore',downcast='integer')
#df=pd.DataFrame(values)
#df=df.convert_objects(convert_dates=False, convert_numeric=True, convert_timedeltas=False, copy=True)
df.shape

[2020-08-12 22:01:55.369000]	Cargando registros...
[2020-08-12 22:01:55.725000]	Finalizada la carga de registros.
{'publisher': 32, 'rights': 1255, 'description': 1255, 'language': 1255, 'format': 1253, 'creator': 1255, 'date': 1254, 'relation': 421, 'source': 1, 'contributor': 831, 'title': 1255, 'identifier': 1255, 'type': 1255, 'subject': 1255}


(1255, 14)

In [14]:
df.head(1)

,contributor,creator,date,description,format,identifier,language,publisher,relation,rights,source,subject,title,type
0,"Rossi, Gustavo Héctor","Garrido, Alejandra",1997,"Las tres primeras secciones de este trabajo plantean el objetivo que se ha perseguido, su motivación y principales contribuciones, las que luego serán fundamentadas en los siguientes capítulos.\r\nLa cuarta sección presenta el estado del arte con respecto a los conceptos básicos sobre los que se ha trabajado, y al mismo tiempo explica qué faceta de cada uno es la que ha incidido en esta tesis. Cabe destacar que se suponen conocimientos básicos sobre el paradigma de orientación a objetos.\r\nLa quinta sección caracteriza las aplicaciones que podrán hacer uso de este trabajo a distintos niveles.",application/pdf,http://sedici.unlp.edu.ar/handle/10915/2161,spa,NaN,NaN,info:eu-repo/semantics/openAccess,NaN,Ciencias Informáticas,OO-Navigator: un framework para hipermedia : Extendiendo aplicaciones orientadas a objetos con funcionalidad de hipermedia,info:eu-repo/semantics/bachelorThesis


## TF-IDF Transform

###  tf-idf-cosine: para encontrar la similitud entre documentos

In [19]:
def processing(file):
    file = file.encode('ascii', 'ignore').decode('ascii')
    #Tokenizing the string object from the text file
    tokens = word_tokenize(file)
    
    #Removing punctuations and making all letters to lower case
    words = [w.lower() for w in tokens if w.isalpha()]
    
    #Removing the stopwords from the list of words
    stop_words = set(stopwords.words('spanish'))
    filtered_tokens = [w for w in words if w not in stop_words]
    
    #Stemming the list of words
    porter = nltk.PorterStemmer()
    stemmed = [porter.stem(t) for t in filtered_tokens]
    
    return ', '.join(str(v) for v in filtered_tokens)

df['abstract'] = df['description'].apply(processing)
doc = df['abstract'] 

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(doc)

#cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
#related_docs_indices = cosine_similarities.argsort()[:topN:-1]

#print related_docs_indices
#print cosine_similarities[related_docs_indices]
result = pd.DataFrame(columns=["id", "abstract", "id similar", "abstract", "similaridad%"])

for i in range(len(doc)):
    cosine_similarities = linear_kernel(tfidf[i:i+1], tfidf).flatten()
    # make pairs of (index, similarity)
    cosine_similarities = list(enumerate(cosine_similarities))
    # delete the cosine similarity with itself
    cosine_similarities.pop(i)
    # get the tuple with max similarity
    most_similar, similarity = max(cosine_similarities, key=lambda t:t[1])
    result.loc[len(result)] = [df['identifier'].iloc[i], df['description'].iloc[i],df['identifier'].iloc[most_similar], df['description'].iloc[most_similar], similarity]

In [27]:
result[(result['similaridad%'] >= 0.9) & (result['similaridad%'] <= 1.0)].sort_values(by=['similaridad%'], ascending=False).head(5)

,id,abstract,id similar,abstract,similaridad%
725,http://sedici.unlp.edu.ar/handle/10915/57803,"En este trabajo se presenta un estudio preliminar de estrategias para el uso de dispositivos móviles, con recarga del acumulador basada en energía solar fotovoltaica. Su empleo está previsto en zonas aisladas que se encuentren fuera del alcance de los centros de distribución de energía. Como primer paso, se estableció la cantidad de energía que consume un dispositivo móvil para ejecutar aplicaciones de uso frecuente en zonas rurales. Luego, tomando como parámetro valores de radiación solar de referencia y sin utilizar baterías externas, se midió la potencia que entrega un panel fotovoltaico de treinta y seis celdas en serie y se verificó que el mismo sea capaz de satisfacer los requerimientos de energía del celular. Finalmente se probaron distintas configuraciones del panel, para lo que se procedió a levantar la capa de Tedlar posterior, soldando contactos en diferentes zonas del mismo. El mayor impacto positivo se consiguió, abriendo el módulo en su cara posterior lo que posibilita conectar dieciocho celdas en serie, en paralelo con otras dieciocho. Los resultados obtenidos en el trabajo experimental fueron comparados para establecer la mejora introducida por el uso de un módulo FV con esta última conexión. La optimización del sistema fotovoltaico incrementó la velocidad de carga de la batería del dispositivo y posibilitó la ejecución de un mayor número de aplicaciones móviles.",http://sedici.unlp.edu.ar/handle/10915/53259,"En este trabajo se presenta un estudio preliminar de estrategias para el uso de dispositivos móviles, con recarga del acumulador basada en energía solar fotovoltaica. Su empleo está previsto en zonas aisladas que se encuentren fuera del alcance de los centros de distribución de energía. Como primer paso, se estableció la cantidad de energía que consume un dispositivo móvil para ejecutar aplicaciones de uso frecuente en zonas rurales. Luego, tomando como parámetro valores de radiación solar de referencia y sin utilizar baterías externas, se midió la potencia que entrega un panel fotovoltaico de treinta y seis celdas en serie y se verificó que el mismo sea capaz de satisfacer los requerimientos de energía del celular. Finalmente se probaron distintas configuraciones del panel, para lo que se procedió a levantar la capa de Tedlar posterior, soldando contactos en diferentes zonas del mismo. El mayor impacto positivo se consiguió, abriendo el módulo en su cara posterior lo que posibilita conectar dieciocho celdas en serie, en paralelo con otras dieciocho. Los resultados obtenidos en el trabajo experimental fueron comparados para establecer la mejora introducida por el uso de un módulo FV con esta última conexión. La optimización del sistema fotovoltaico incrementó la velocidad de carga de la batería del dispositivo y posibilitó la ejecución de un mayor número de aplicaciones móviles.",1.000000
740,http://sedici.unlp.edu.ar/handle/10915/53259,"En este trabajo se presenta un estudio preliminar de estrategias para el uso de dispositivos móviles, con recarga del acumulador basada en energía solar fotovoltaica. Su empleo está previsto en zonas aisladas que se encuentren fuera del alcance de los centros de distribución de energía. Como primer paso, se estableció la cantidad de energía que consume un dispositivo móvil para ejecutar aplicaciones de uso frecuente en zonas rurales. Luego, tomando como parámetro valores de radiación solar de referencia y sin utilizar baterías externas, se midió la potencia que entrega un panel fotovoltaico de treinta y seis celdas en serie y se verificó que el mismo sea capaz de satisfacer los requerimientos de energía del celular. Finalmente se probaron distintas configuraciones del panel, para lo que se procedió a levantar la capa de Tedlar posterior, soldando contactos en diferentes zonas del mismo. El mayor impacto positivo se consiguió, abriendo el módulo en su cara posterior lo que posibi

In [25]:
result[(result['similaridad%'] > 0.7) & (result['similaridad%'] < 0.8)].sort_values(by=['similaridad%'], ascending=False).head(5)

,id,abstract,id similar,abstract,similaridad%
1174,http://sedici.unlp.edu.ar/handle/10915/84612,"Diversos proyectos militares han centrado sus esfuerzos en diseñar un sistema digital integrado como soporte para la toma de decisiones del personal militar, para afrontar el reto de poseer un Conciencia Situacional (CS) degradada. En este trabajo se propone un framework de Realidad Aumentada (RA) denominado RAIOM (Realidad Aumentada para la Identificación de Objetivos Militares) –entendiéndose por identificación de objetivos militares a todos los objetivos militares del enemigo circundantes a los soldados que representan una amenaza bélica- cuyo objetivo es mejorar la CS de los soldados en el campo de batalla mediante el uso de la RA. Para el desarrollo del framework se consideró como principal característica a la CS, y a la RA como tecnología de visualización e interpretación contextual. La aplicación de metodologías tales como SCRUM y Diseño Centrado en el Usuario (DCU) permitió desarrollar los despliegues correspondientes. En particular, este trabajo presenta una recopilación actualizada de diferentes sistemas utilizados en el ámbito militar y se analizan las características principales que debe poseer un framework de software basado en RA para uso militar. Se diseña un framework basado en RA que permite la visualización de objetivos tácticos tales como visualización de objetivos tácticos en un radar en 360 grados, posicionamiento y orientación espacial de los objetos y del operador, ayudas visuales mediante iconografía/simbología adaptable, uso de menús interactivos mediante reconocimiento gestual y vocal, reconocimientos de objetos edilicios y uso de cartografía en 3D mediante uso de RA. Se utiliza una arquitectura basada en capas para facilitar la integración tecnológica como también una arquitectura de sistemas distribuida donde la principal característica se centra en realizar el procesamiento de imágenes en un componente de hardware externo. Se realizan los despliegues utilizando el framework de RA propuesto en el presente trabajo y, finalmente, se arriba a las conclusiones finales y a las futuras líneas de investigación.",http://sedici.unlp.edu.ar/handle/10915/86707,"Durante las operaciones militares, los campos de batalla se convierten en zonas fracturadas donde el nivel de confusión, el ruido y la ambigüedad impactan en la manera de alcanzar los objetivos tácticos. La Conciencia Situacional (CS) se convierte en un reto ya que la percepción de la situación es inestable, lo que conduce a la comprensión degradada y a la incapacidad del soldado en proyectar los resultados apropiados. Para afrontar dicho reto diversos proyectos militares han centrado sus esfuerzos en diseñar un sistema digital integrado como soporte para la toma de decisiones del personal. \r\nEn esta tesis doctoral se propone un framework de Realidad Aumentada (RA) denominado RAIOM (Realidad Aumentada para la Identificación de Objetivos Militares) –entendiéndose por identificación de objetivos militares a todos los objetivos militares del enemigo circundantes a los soldados que representan una amenaza bélica- cuyo objetivo es mejorar la CS de los soldados en el campo de batalla mediante el uso de la RA. Para el desarrollo del framework se consideró como principal característica a la CS, y a la RA como tecnología de visualización e interpretación contextual. La aplicación de metodologías tales como SCRUM y Diseño Centrado en el Usuario (DCU) permitió identificar y analizar los requisitos funcionales y desarrollar los despliegues correspondientes. En particular, este trabajo presenta una recopilación actualizada de diferentes frameworks, menciona las características principales que debe poseer un framework de software de RA para uso militar, se detalla el uso de metodologías combinadas, se considera una arquitectura basada en capas para facilitar la integración tecnológica como también una arquitectura de sistemas distribuida donde la principal característic

###  Implementar clustering de k-medias usando similitud de coseno

In [28]:
spanish_stopwords = stopwords.words('spanish')

def get_tfidf_matrix(X):
    #function to calculate term freq for each element
    tf = lambda x: np.log(1+x)
    # apply tf lambda on matrix
    tf_result = np.apply_along_axis(tf,1,X.toarray())  
    #Get the total number of documents/reviews
    N = len(X.toarray())
    # Get frequency of each word across all documents
    overall_freq = X.toarray().sum(axis=0)
    # function to calculate idf for each word
    idf = lambda x: np.log(N/(1+x))
    idf_result = idf(overall_freq)
    #multiply tf with idf for each document
    return np.multiply(tf_result,idf_result)

vectorizer = CountVectorizer(stop_words=spanish_stopwords)
X = vectorizer.fit_transform(df.description.values)
tf_idf = get_tfidf_matrix(X)

class Kmeans:

    def __init__(self, k, seed = None, max_iter = 200):
        self.k = k
        self.seed = seed
        if self.seed is not None:
            np.random.seed(self.seed)
        self.max_iter = max_iter
        
            
    # randomly Initialise Centroids
    def random_centres(self, data):
        initial_centres = np.random.permutation(data.shape[0])[:self.k]
        self.centres = data[initial_centres]

        return self.centres
    
    # get  distance of data from clusters and assign data point to closest cluster.
    def assign_clusters(self, data):
        if data.ndim == 1:
            data = data.reshape(-1, 1)
        dist_to_centres =  self.cosine_sim(data, self.centres)
        self.cluster_val = np.argmax(dist_to_centres, axis = 1)
        return  self.cluster_val
    
    # get the cosine similarity of two vectors
    def cosine_sim(self, X, Y):
        dot_product = np.dot(X, Y.T)
        review_norm = np.linalg.norm(X, axis = 1)
        review_norm = review_norm.reshape(X.shape[0],1)
        center_norm = np.linalg.norm(Y, axis = 1)
        center_norm = center_norm.reshape(Y.shape[0],1)
        return dot_product / (review_norm.dot(center_norm.T))
        
    # get average of all data points in cluster and assigns new centres 
    def update_centres(self, data):
        self.centres = np.array([data[self.cluster_labels == i].mean(axis = 0) for i in range(self.k)])
        return self.centres
    
    # Predict which cluster data point belongs to
    def get_prediction(self, data):
        return self.assign_clusters(data)
    
    # assign all data to 2nd cluster
    def assign_to_random_clusters(self, data):
        self.cluster_labels = np.ones((data.shape))
        
    #  main loop to fit the algorithm       
    def fit_kmeans(self, data):
        #print(self)
        self.centres = self.random_centres(data)
        self.assign_to_random_clusters(data)
        
        # main kmeans loop
        for iter in range(self.max_iter):
            self.cluster_labels = self.assign_clusters(data)
            self.centres = self.update_centres(data) 
            if iter % 5 == 0:
                print("Model Iteration %d " %iter)
        print("Cluster labels:::*********")
        #print(self.cluster_labels)
        #print(type(self.cluster_labels), self.cluster_labels.shape)
        print("Model finished running")
        return self
    
def print_cluster(label, n):
    df["cluster_label"] = list(labels)
    for cluster in range(n):
        print("Cluster:", cluster)
        display(df[df.cluster_label == cluster][["description","cluster_label"]].head(5))

In [30]:
# Cluster = 15
n = 15
#Generate 5 random indices 
np.random.seed(45678)
indices = np.random.choice(tf_idf.shape[0], n, replace=False)
cluster_centres = tf_idf[indices]

np.set_printoptions(threshold=sys.maxsize)
km = Kmeans(n, seed= 1234, max_iter=35)
labels = km.fit_kmeans(tf_idf).cluster_labels
print_cluster(labels, n)

Model Iteration 0 
Model Iteration 5 
Model Iteration 10 
Model Iteration 15 
Model Iteration 20 
Model Iteration 25 
Model Iteration 30 
Cluster labels:::*********
Model finished running
('Cluster:', 0)


,description,cluster_label
106,"En este trabajo propongo una formalización basada en gramáticas de atributos que permite capturar en forma precisa la estructura de los diagramas y su simultánea evaluación (p. ej., construir/evaluar automáticamente el modelo semántico de los diagramas durante la construcción de los mismos). Comencé este trabajo en el marco de un proyecto colaborativo entre la UNLP y la PUC (Pontificia Universidad Católica do Rio de Janeiro) y reporté algunos resultados preliminares. En dicha instancia introduje la idea básica do utilizar gramáticas para capturar la estructura de los diagramas y estudié además distintas alternativas para la formalización. Además, ese proyecto culminó con la construcción de un prototipo funcional que mostraba la importancia práctica de la investigación previamente mencionada.\r\nEn esta tesis reporto el trabajo mencionado y brindo un análisis más comprensivo y extenso del uso de gramáticas para la descripción de los lenguajes visuales. Por otro lado, y a fin de demostrar la importancia práctica, desarrollo un editor que utiliza los conceptos expresados, mostrando de esta forma cómo el diseño y prototipación de lenguajes visuales se vé simplificada en forma significativa. Además el trabajo está acompañado de una serie de ejemplos prácticos que apoyan el desarrollo.",0
228,"Esta tesis investiga posibles incrementos de la performance en las soluciones de ciertos problemas de optimización combinatorial NP-duros. Ejemplos de esto son los problemas de secuenciamiento puros. Se realiza una reseña de los métodos convencionales, comparándolos con los pertenecientes al campo de la Computación Evolutiva, conjuntamente con la propuesta de eventuales mejoras a estos últimos. Las aplicaciones prácticas discutidas en esta tesis se encuentran fuertemente relacionadas con la administración, el diseño de redes en general, y el diseño de circuitos.",0
364,"In this book, Hansen presents a collection of structured multicomputer programs for computational science, with the unifying idea of programming paradigm.",0
409,"The main purpose of this book is to present the contents of an introductory course in databases. In this regard, one can find a number of material validity and recognized different scopes. This work is developed under the guidelines that the authors use in dictating the course Introduction to Databases, Faculty of Informatics, National University of La Plata, Argentina",0
448,"A social network is formed by a set of actors and the relationships established by them. SNA leads to distinct goals and perspectives of social network analysis and computer science. This paper introduces the study of social networks and their relationship with trust. We study the methods of detection and description of structural properties. This covers the concepts, methods and data analysis techniques of social networks analysis. After that, we introduce the concept of trust and its relation with social networks, because trust is a crucial factor in the interaction between actors. We use the Node XL tool and a case study in a classroom environment, to test the usability of SNA techniques considering metrics to measure trust and, significantly, the power of a tightly integrated metrics/visualization tool to spark insight and facilitate sense making to SNA.",0


('Cluster:', 1)


,description,cluster_label
6,"Las herramientas de simulación a eventos discretos ofrecen una amplia gama de opciones para realizar análisis detallados en base a los resultados arrojados luego de las corridas de cada simulación. En este trabajo proponemos una alternativa en la cual se persisten todas las entidades del modelo simulado, lo cual sienta las bases para el desarrollo de nuevas herramientas de análisis que permitan conocer en un alto nivel de detalle la evolución de la simulación como un todo y de cada una de las entidades por separado. El desarrollo, basado en el lenguaje de simulación GPSS, consta de todo el modelo que éste lenguaje utiliza para ejecutar simulaciones y de un conjunto de entidades básicas que permiten crear verdaderas simulaciones. Este trabajo también incluye una herramienta para ejecutar simulaciones y conocer las historias de las entidades involucradas en las mismas.",1
32,"El trabajo que se presenta es la última evolución conseguida hasta el momento de un ambiente de simulación donde se modelizan e implementan situaciones en la que una BDD debe mantener la integridad de la información ante fallos producidos durante la ejecución de transacciones, enfocándose en la utilización y posterior comparación de diferentes protocolos de commit atómicos (ACP). La implementación se realizó en Java debido a diferentes aspectos como facilidad de trabajo, portabilidad, etc. Las situaciones evaluadas en el ambiente se basan en la recuperación de fallos de transacciones en un entorno de datos distribuidos, permitiendo seleccionar, para el desarrollo de la simulación, entre los protocolos dos fases, de tres fases, el protocolo de presumed commit y presumed abort. A partir del desarrollo y los resultados obtenidos se presentan una serie de conclusiones respecto de las evaluaciones registradas.",1
56,"Las redes neuronales artificiales han demostrado ser exitosas resolviendo adecuadamente muchos problemas, pero como todo paradigma también presentan debilidades, tales como dificultad para hallar una topología adecuada, dificultad para elegir el valor apropiado de algunos parámetros del entrenamiento, fuerte dependencia con los valores iniciales de los pesos escogidos aleatoriamente, a veces requieren funciones de transferencia derivables, y entre otras más, pueden caer mínimos locales de la función de error que desean minimizar.\r\nEstas dificultades suelen resolverse a fuerza de la experiencia y habilidades de los expertos humanos. Ahora bien, la Computación Evolutiva, que junto al Conexionismo de las redes neuronales artificiales conforma la rama subsimbólica de la Inteligencia Artificial, puede mejorar estos resultados. Así es posible reemplazar el entrenamiento tradicional de las redes neuronales artificiales por la aplicación de los principios darwinianos –evolución por selección natural– a las redes neuronales artificiales debidamente codificadas en un cromosoma. Esto resuelve algunas de las debilidades mencionadas y da origen al nuevo paradigma denominado Neuroevolución.\r\nLos investigadores del área han propuesto distintas estrategias dentro de este paradigma. Entre otras cosas se utilizan los algoritmos evolucionarios para conseguir la arquitectura de la red, los pesos de las conexiones y las funciones de transferencia de los nodos. Incluso se han propuesto hibridaciones combinando aprendizaje evolucionario con las técnicas de entrenamiento propias de las redes neuronales artificiales.\r\nEsta tesina se enmarca dentro del paradigma de la Neuroevolución y, si bien me he ajustado a la propuesta de trabajo de grado presentada, aspiro a tratar unos cuantos conceptos propios de esta renovada rama de la Inteligencia Artificial.",1
124,"El campo de acción de las aplicaciones hypermedia, en especial el de las aplicaciones web, se encuentra en constante evolución. Muchas son las razones y entre las más importantes se destacan el avance de las tecnologías y su incorporación a innumerables disciplinas. Los ejemplos sobr

('Cluster:', 2)


...

,description,cluster_label
1,"Si bien ambos modelos [modelo en cascada y prototipación][ ofrecen una buena ayuda para el desarrollo de soft, tienen el inconveniente de que dicen “que” es lo que hay que hacer, pero sin decir el “como”. Este trabajo tiene por objetivo plantear el “como” para la fase de análisis de requisitos y diseño preliminar, en un ambiente de objetos. Se refinan métodos y heurísticas del modelo del baseline tratado en [Leite 90], [Leite 93], [Leite 95b] y [Leite 97c], Y se implementa una herramienta que soporta el modelo presentado.\r\nLa organización del trabajo es la siguiente: en los puntos 1.1.2, 1.1.3 y 1.1.4 se da una idea introductoria y global de todo el modelo. En los puntos 1.2, 1.3 y 1.4 se tratan en profundidad cada uno de los pilares del modelo: LEL, Escenario y clases candidatas, respectivamente. Y por último, en la sección 2 se trata la herramienta que automatiza el modelo presentado.",12
3,"El aporte más importante de este trabajo es la definición de una arquitectura que permite la definición y el tratamiento de un campo continuo como un objeto del sistema; esto significa: \r\n• El campo continuo se define como un conjunto finito de pares (posición, valor) conocido a través de algún método de toma de datos (la muestra origen) más un método de estimación necesario para calcular puntos intermedios que no han sido relevados. \r\n• Se pueden asociar distintos algoritmos de estimación a una muestra y, en el caso necesario, cambiar dicho algoritmo dinámicamente, de manera transparente a la aplicación subyacente. \r\n• Es posible definir y asociar distintos métodos de implementación para una muestra determinada. \r\n• El encapsulamiento del campo continuo permite utilizarlo como un objeto en sí mismo o asociar valores del campos a objetos (discretos) a través de atributos. \r\nEn el capítulo 2 se describen las principales características de los sistemas de información geográfica. Además de considerar cúales son los principales elementos de estos sistemas, se detallan los diferentes procesos que aparecen en el desarrollo y utilización de los sistemas en particular: \r\n• la entrada de datos, la cual tiene una importancia vital en el proceso, pues ésta determinada en gran parte cúan correcta (y por lo tanto cúan confiable) es la información que se está manipulando. \r\n• La manipulación de los datos se describe a partir de las funciones de análisis que utilizan la mayoría de los productos de SIG existentes en el mercado. \r\n• La definición de las interfaces que definen la salida de información y el análisis que se puede realizar sobre los datos geográficos. \r\nEn el capítulo 3 se detallan las estructuras de datos que se utilizan para soportar información geográfica y se plantean discusiones sobre el uso de cada una. \r\nEn el capítulo 4 se describen los conceptos básicos del modelo de objetos que se utiliza como base para la definición de la arquitectura para manipular campos continuos. Este modelo está basado en la tecnología de “Patrones de Diseño” [Gamma et al.95] la cual se describe sucintamente incluyendo un ejemplo. \r\nEn el capítulo 5 se introducen los conceptos básicos de un campo continuo y las operaciones que habitualmente se requieren sobre este tipo de datos. Se discuten los problemas que existen actualmente para la representación y manipulación de los mismos y se describen algunas investigaciones referentes al tema. \r\nEn el capítulo 6, se describe la arquitectura definida para campos continuos sobre la base del modelo descripto en el capítulo 4. Se detalla la arquitectura y se especifican las ventajas de su definición. Para mayor claridad se define un ejemplo en el que se muestra su uso. \r\nEn el capítulo 7 se presenta una implementación posible para la arquitectura definida en el capítulo 6. \r\nEn el capítulo 8 se describen algunos trabajos relacionados que utilizan técnicas de la orientación a objetos para modelizar aplicaciones SIG. También se hace una discusión sobre las ventaj

('Cluster:', 13)


,description,cluster_label
34,"El presente trabajo de grado surgió para canalizar inquietudes respecto del uso de la computadora en el aprendizaje. El objetivo inicial tuvo su origen en la idea de realizar un aporte para lograr mejorar el nivel de lectura y escritura de los estudiantes secundarios, en una forma amena, ágil, con un medio atractivo o usual para ellos, como es la computadora; mediante algunos de los mejores textos literarios de ficción de habla española, su análisis e interpretación. Las técnicas que se han utilizado para llevar a cabo desarrollos informáticos orientados al aprendizaje han sido diversas, siendo la hipermedia una de las que goza de mayor aceptación. Esta tecnología proporciona características que resultan muy útiles en el campo del aprendizaje por computadora, como son la interactividad, el uso de grandes bases de información, la información multimedia y la representación del conocimiento de forma similar a la forma de procesamiento de la información del alumno.",13
46,"Se está haciendo un considerable progreso en la integración de los sistemas de información espacial, GIS y los modelos matemáticos que rigen en medioambiente. Para la mayoría de los proyectos de modelos ambientales, los GISs son vistos como bases de datos apropiadas y bien estructuradas para el manejo de grandes cantidades de datos espaciales. La herramientas GIS tradicionales, como los overlays y buffering son también importantes para el desarrollo de juegos de datos derivados (derivative datasets) que sirven para generar vistas sobre variables. Muchos expertos esperan que pronto se incorporen mejores métodos de análisis espacial a los GIS actuales, la tecnología GIS se convertirá en una herramienta importante en todos los aspectos del modelamiento, incluyendo construcción de modelos, validación y operación. Sin embargo, existen grandes incompatibilidades que obstruyen la verdadera integración. Los GIS manejan datos estáticos y discretos mientras los modelos ambientales tratan fenómenos dinámicos y continuos.\r\nLas BD GIS manipulan datos en términos de ubicación, distribución y relaciones espaciales, mientras que los modelos ambientales trabajan con la transferencia masa y energía. Para lograr una integración total de ambos necesitamos agregar el dinamismo y la continuidad a nuestro concepto de datos espaciales; e interacción espacial y funcionalidad a los modelos ambientales.\r\nEste trabajo de investigación apunta a la primera de estas necesidades, considerando lo que implica trabajar con fenómenos continuos directamente en el contexto de GIS. Se tendrá en cuenta cómo los fenómenos espacialmente distribuidos se caracterizan en los modelos ambientales, una estrategia para trabajar con dichos fenómenos de manera que puedan ser eficientes y convincentemente incorporados en aplicaciones GIS.\r\nYa que la mayoría de los fenómenos ambientales tratan con fenómenos que son continuos en el espacio, es necesario proveer formas en las que los modeladores puedan trabajar directamente con los fenómenos continuos, en vez de verse forzados a trabajar con datos espaciales discretos.\r\nDebemos reconocer que los datos espaciales son representaciones de la realidad, no la realidad en sí, por lo que deberíamos concentrarnos en lograr un análisis que no dependa de la representación, sino de la realidad en sí (frame free analysis).",13
64,"Un proceso efectivo proporciona normas para la realización de aplicaciones eficientes y de calidad. Además guía a los distintos participantes (desarrolladores, clientes y usuarios) de un proyecto.\r\nEl método debe ayudarnos a conseguir aplicaciones con características tales como extensibilidad, cambiabilidad y reusabilidad y a descubrir, tempranamente, los riesgos que pueden llevar al fracaso del software.\r\nEl proceso de desarrollo de software está compuesto por varias actividades diferentes. La pregunta que nos surge es: ¿cómo deben organizarse estas actividades para lograr la calidad deseada en el producto f

('Cluster:', 14)


,description,cluster_label
14,La primera parte de este trabajo tiene como objetivo describir la especificación de CORBA (Common Object Request Broker Arquitecture) en general. En la segunda parte veremos cómo CORBA se Integra con Java. La Tercera parte incluye la comparación de CORBA con otros sistemas que permiten construir aplicaciones Client/Server y se perfilan como sus competidores.\r\nComo apéndice presentaré un ejemplo práctico del uso de CORBA con JAVA a través de una aplicación.,14
24,"Un caching proxy server colecciona todas las páginas que pasan a través de él, así cuando un usuario pregunta por una página, el proxy server, si la tiene, la retoma de su cache, sin necesidad de acceder al servidor real. De ésta manera se ahorra ancho de banda y se agilizan las respuestas.\r\nCada requerimiento que arriba a un proxy ó a un servidor, se almacena en archivos de logs. Existen varios tipos de archivos de logs, que se utilizan para auditoría del rendimiento, monitoreo, control de errores, seguridad y estadísticas.\r\nAsí, existen actualmente muchos analizadores de logs de Web, que permiten realizar un estudio detallado de un sitio, ayudando a la planificación y a la auditoría de seguridad.\r\nEn éste trabajo, se plantea la problemática de no actualización de los logs de servidores cuando está de por medio un servidor proxy, ya que en éste esquema, el log del servidor no refleja la realidad de accesos al mismo.\r\nEsto ocurre, porque cuando un servidor proxy retoma documentos al cliente desde su cache local, dichos movimientos quedan registrados en el log del proxy y no en el log del servidor.\r\nPor éste motivo, la información recolectada por los analizadores de logs no es exacta y se distorsiona, provocando estadísticas no tan precisas.\r\nComo solución, la idea es mantener el archivo de log de accesos del servidor actualizado, para lo cuál se necesita intercambiar información entre logs de proxies y logs de servidores, es decir, lograr una sincronización de los mismos.\r\nSe proponen aquí dos modelos para resolver la problemática enunciada, y se analizan los pro y los contra de cada uno. Así, se adoptará alguno cómo solución, o ambos como dos métodos alternativos.",14
35,"Este proyecto se concentró especialmente en los métodos utilizados para la captura y filtrado de paquetes, y para ello, el primer paso fue la especificación de la plataforma seleccionada: sistemas Unix o basados en Unix (sistemas Linux). Esta selección se fundamentó, principalmente, por su performance, confiabilidad y alta disponibilidad de herramientas que dichas plataformas soportan. La flexibilidad de estos sistemas es aún mayor si se considera que el código fuente de algunos de ellos es abierto (como es el caso de Linux). Por otra parte, las herramientas de monitoreo, requieren determinadas características que estos sistemas pueden brindar (manejo de threads, manejo de tareas en forma automática mediante scripts,…).\r\nEl control de procesos provisto por los sistemas basados en Unix incluye la creación de nuevos procesos, la ejecución de programas y la terminación de procesos. Contando solamente con lo recién mencionado, la única manera de que los procesos intercambien información es transfiriéndola a través de archivos abiertos mediante las llamadas al sistema fork o exec, o a través del sistema de archivos (file system). En esta implementación se utilizó la segunda forma de comunicación (accediendo a los archivos mediante funciones del sistema de archivos).",14
36,"La idea principal de nuestro proyecto es la implementación de un Lector de Noticias online/offline, desarrollado en lenguaje Java. Para llevar a cabo nuestro desarrollo tuvimos como objetivo de implementación el cumplimiento de las carácterísticas básicas de la mensajeria dentro de Usenet, la integración de dos importantes servicios de internet, mail y newsreader y la implementación de una interfaz gráfica que permita visualmente guiar al usuario por las distintas funcionalidades del lec